## Data Visualization

As a data scientist, you likely work through data exploration processes on nearly every project. Exploratory data analysis can entail many things from finding relevant data and cleaning it to running analysis and building models. The ability to visually analyze and interact with data is key during the exploratory process and the final presentation of insights.

With that in mind, this guide introduces the basic building blocks for creating web-based, dynamic, and interactive map visualizations inside of a Jupyter Notebook with CARTOframes.

In this guide, you are introduced to the Map and Layer classes, how to explore data with Widgets and Popups, how to use visualization styles to quickly symbolize thematic attributes and options for creating maps to share your findings. You can check also the [Data Visualization examples](/developers/cartoframes/examples/#example-data-visualization).

### Visualize data from a file

CARTOframes allows creating maps directly with data from GeoJSON or CSV files (with geometric data).

In [1]:
from geopandas import read_file

points_gdf = read_file('http://libs.cartocdn.com/cartoframes/samples/starbucks_brooklyn_geocoded.geojson')

In [2]:
from cartoframes.viz import Layer

Layer(points_gdf)

If the source, instead of a GeoJSON, is a CSV file with a column containing the geometry encoded in WKB/WKT, you can pass the name of the column as `geom_col` directly to the Layer and the geometry is automatically decoded.

In [3]:
from pandas import read_csv

polygons_df = read_csv('http://libs.cartocdn.com/cartoframes/samples/starbucks_brooklyn_iso_enriched.csv')
polygons_gdf = read_file('http://libs.cartocdn.com/cartoframes/samples/starbucks_brooklyn_iso_enriched.geojson')

In [4]:
from cartoframes.viz import Map

Map([
    Layer(polygons_df, geom_col='the_geom'),
    Layer(points_gdf)
])

### Visualize data from CARTO

For big datasets visualization (>30MB), we recommend rendering layers directly from a CARTO account. This, in addition to improving the performance, allows you to apply SQL queries to the tables stored in CARTO.

_Note: You'll need [CARTO Account](https://carto.com/signup) credentials to perform this action._

In [5]:
from cartoframes.auth import set_default_credentials

set_default_credentials('cartoframes')

In [6]:
Map([
    Layer('countries'),
    Layer('SELECT * FROM global_power_plants WHERE capacity_mw > 1000'),
    Layer('world_rivers')
])

### Map configuration

The map basemap and initial viewport can be customized. CARTO provides out-of-the-box basemaps: `positron`, `voyager` and `darkmatter`, but it supports also any URL to load external basemaps.

In [7]:
from cartoframes.viz import basemaps

Map(
    basemap=basemaps.voyager,
    viewport={'zoom': 13.5, 'lat': 40.4353, 'lng': -79.9916}
)

### Data-driven styling

CARTOframes provides out of the box style helpers for different kinds of data-driven visualizations:
- Basic
- Color: bins, category, continuous
- Size: bins, category, continuous
- Animation

Style helpers contain built-in cartography styling. However, each method provides lots of parameters to adjust your visualization to your needs (color, size, stroke, color palette, classification methods, etc.). There are also legends, popups, and widgets. By default, only the legend and popup hover are enabled.

In [8]:
from cartoframes.viz import size_continuous_style

Layer(
    points_gdf,
    size_continuous_style('revenue')
)

In [9]:
from cartoframes.viz import Layer, color_bins_style

Layer(
    'eng_wales_pop',
    color_bins_style('pop_sq_km', palette='mint', bins=5, opacity=0.8),
    default_widget=True,
    default_popup_hover=False
)

### Legends

Legends are aligned with style helpers. Each style helper contains what we call the "default legend" based on the style:
- Basic
- Color: bins, category, continuous
- Size: bins, category, continuous

Depending on the geometry of the visualization (points, lines, polygons), the legend will render the corresponding symbology. Legends are also customizable, you can edit the title, or add a description or a footer.

When having multiple layers in a map, legends will stack according to the order of the layers.

In [10]:
from cartoframes.viz import size_continuous_style, size_continuous_legend  # or just default_legend

Layer(
    points_gdf,
    size_continuous_style('revenue', size_range=[10,50], stroke_color='turquoise', stroke_width=2, opacity=0),
    legends=size_continuous_legend(
        title='Annual Revenue',
        description='Revenue in dollars ($)',
        footer='Source: <a href="https://www.starbucks.com/">Starbucks</a>'
    )
)

### Interactive widgets

There are several widgets:
- Formula
- Histogram
- Category
- Time-series
- Animation

Widgets are two-way interactive: when the widgets are filtered they affect the visualization and the other filters, and vice versa, when the viewport changes, the widgets show the information filtered by viewport (unless the flag is_global is set).

In [11]:
from cartoframes.viz import formula_widget, category_widget, histogram_widget

Layer(
    polygons_gdf,
    widgets=[
        formula_widget('popcy', 'sum', 'Population'),
        formula_widget('inccymedhh', 'sum', 'Median Income Average', is_global=True),
        histogram_widget('educybach', 'Education'),
        category_widget('id_store', 'Store ID')
    ]
)

### Click and hover popups

Popups are a nice tool the handy explore your data in a map. CARTOframes provides full customization to both click and hover popups. Using the helper `popup_element` you can display any property from your data.

When having multiple layers in a map, popups will stack according to the order of the layers.

In [12]:
from cartoframes.viz import popup_element

Layer(
    polygons_gdf,
    popup_hover=[
        popup_element('popcy', 'Population')
    ],
    popup_click=[
        popup_element('popcy', 'Population'),
        popup_element('id_store', 'Store ID'),
        popup_element('inccymedhh', 'Median Income'),
        popup_element('lbfcyempl', 'Employed Population'),
    ]
)

### Grid layout

The Layout class provides the feature of showing different maps to, for example, compare variables for the same visualization. These grid maps can also be rendered as static setting the `is_static` parameter.

In [13]:
from cartoframes.viz import Layout

Layout([
    Map(Layer(polygons_gdf, style=color_bins_style('inccymedhh', bins=7, palette='mint', stroke_width=0))),
    Map(Layer(polygons_gdf, style=color_bins_style('popcy', palette='pinkyl', stroke_width=0)))
], 2, 1, map_height=400, is_static=True)

### Publish and share

CARTOframes allows to publish visualizations rendering local data (GeoDataFrame) and also public and private tables from CARTO accounts. The `Map` class provides a method to publish a map in a static URL to share it with others. Maps can be published also in "protected" mode using a password in case you don't want to make it public.

_Note: You'll need [CARTO Account](https://carto.com/signup) credentials to perform this action._

In [14]:
from cartoframes.auth import set_default_credentials

set_default_credentials('creds.json')

In [15]:
map_viz = Map(Layer('countries'))

In [16]:
map_viz.publish(
    name='countries',
    password='1234',
    if_exists='replace')

{'id': 'dcb4ebde-998c-4f98-b44d-ae790c59845e',
 'url': 'https://cartoframes-org.carto.com/u/cartoframes/kuviz/dcb4ebde-998c-4f98-b44d-ae790c59845e',
 'name': 'countries',
 'privacy': 'password'}